## TensorFlow2.0 可视化

In [7]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, optimizers, Sequential,metrics
import datetime
from matplotlib import pyplot as plt
import io


In [8]:
# 编写数据集预处理函数
def preprocess(x, y):
    # 把x转换为 tf.float32类型，并压缩到 [0, 1] 之间，注意：此处除以 255. (包括小数点)
    x = tf.cast(x, dtype=tf.float32) / 255.
    # 只需把y转换为 tf.int32即可
    y = tf.cast(y, dtype=tf.int32)
    return x, y

In [10]:
def plot_to_image(figure):
    '''
    Converts the matplotlib plot specified by 'figure' to a PNG image and returns it. The supplied 
    figure is closed and inaccessible after this call.
    '''
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)    
    return image
def image_grid(images):
    '''Return a 5*5 grid of the MNIST images as a matplotlib figure'''
    # Create a figure to contain the plot.
    figure = plt.figure(figsize=(10, 10))
    for i in range(25):
        # start next subplot.
        plt.subplot(5, 5, i+1, title='name')
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i], cmap=plt.cm.binary)
        
    return figure

batchsz = 128
(x, y), (x_val, y_val) = datasets.fashion_mnist.load_data()
db = tf.data.Dataset.from_tensor_slices((x, y))
db = db.map(preprocess).shuffle(60000).batch(batchsz).repeat(10)

db_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
db_val = db_val.map(preprocess).batch(batchsz, drop_remainder=True)


network = Sequential([   
    layers.Dense(256, activation=tf.nn.relu),    # [b, 784] => [b, 256]
    layers.Dense(128, activation=tf.nn.relu),    # [b, 256] => [b, 128]
    layers.Dense(64, activation=tf.nn.relu),     # [b, 128] => [b, 64]
    layers.Dense(32, activation=tf.nn.relu),     # [b, 64] => [b, 32]
    layers.Dense(10)                             # [b, 32] => [b, 10]
])

# network = Sequential([   
#     layers.Dense(684, activation=tf.nn.relu),    
#     layers.Dense(654, activation=tf.nn.relu),    
#     layers.Dense(624, activation=tf.nn.relu),    
#     layers.Dense(594, activation=tf.nn.relu),    
#     layers.Dense(40, activation=tf.nn.relu),    
#     layers.Dense(20, activation=tf.nn.relu),
#     layers.Dense(15, activation=tf.nn.relu),
#     layers.Dense(10)                           
# ])


network.build(input_shape=(None, 28*28))
network.summary()

optimizer = optimizers.Adam(lr=0.001)

######################################
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = '/home/kukafee/workspace/TensorBoard/' + current_time
# 创建writer
summary_writer = tf.summary.create_file_writer(log_dir)
######################################

# get x from (x, y)
sample_img = next(iter(db))[0]
# get first image instance 
sample_img = sample_img[0]
sample_img = tf.reshape(sample_img, [1, 28, 28, 1])
with summary_writer.as_default():
    tf.summary.image("Training sample:", sample_img, step=0)
    
for step, (x, y) in enumerate(db):
    with tf.GradientTape() as tape:
        x = tf.reshape(x, (-1, 28*28))
        out = network(x)
        
        y_onehot = tf.one_hot(y, depth=10)
        loss = tf.reduce_mean(tf.losses.categorical_crossentropy(y_onehot, out, from_logits=True))
        
    grads = tape.gradient(loss, network.trainable_variables)
    optimizer.apply_gradients(zip(grads, network.trainable_variables))
    
    if step % 100 == 0:
        print(step, 'loss:', float(loss))
        with summary_writer.as_default():
            tf.summary.scalar('train-loss', float(loss), step=step)
            
        # evaluate
        total_correct = 0
        total_num = 0
        for x, y in db_val:
            x = tf.reshape(x, [-1, 28*28])
            out = network(x)  # [b, 10]
            pred = tf.cast(tf.argmax(out, axis=1),tf.int32)
            correct = tf.equal(pred, y)
            correct = tf.reduce_sum(tf.cast(correct, dtype=tf.int32))

            total_correct += int(correct)   # 使用int() 函数把tensor类型转化为numpy的int类型
            total_num += x.shape[0]    # x.shape[0] 是测试集中测试样本的个数

        acc = total_correct / total_num * 100
        print('test acc: ',acc,'%')
        
        print(x.shape)
        val_images = x[:25]
        val_images = tf.reshape(val_images,[-1, 28, 28, 1])
        with summary_writer.as_default():
            tf.summary.scalar('test-acc', float(total_correct/total_num), step=step)
            # 单个显示
            tf.summary.image("val-onebyon-images:", val_images, max_outputs=25, step=step)
            # 在一个画板上一起显示多个图片
#             val_images = tf.reshape(val_images, [-1, 28, 28])
#             figure = image_grid(val_images)
#             tf.summary.image('val-images: ', plot_to_image(figure), step=step)
            
    
    

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             multiple                  200960    
_________________________________________________________________
dense_11 (Dense)             multiple                  32896     
_________________________________________________________________
dense_12 (Dense)             multiple                  8256      
_________________________________________________________________
dense_13 (Dense)             multiple                  2080      
_________________________________________________________________
dense_14 (Dense)             multiple                  330       
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________
0 loss: 2.3940629959106445
test acc:  17.047275641025642 %
(128, 784)
100 loss: 0.5174907445907593
test acc:  79.5